In [1]:
import os
import pandas as pd

from entity.LightNode import LightNode
from utils.DataLoader import DataLoader
from utils.Settings import Setting
from utils.ProjectPath import ProjectPath
from entity import LightCluster, LightNode

path = ProjectPath()
setting = Setting()
dataloader = DataLoader()

cluster_path = path.univ2_cluster_path
cluster_file_template = "cluster_{}.csv"
suspicious_file_template = "cluster_{}_suspicious_nodes.csv"

ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
LOAD END NODES
LOAD CREATION INFO


/home/dustin/Projects/SerialScammer/main/utils/DataLoader.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens["token0"] = tokens["token0"].str.lower()
/home/dustin/Projects/SerialScammer/main/utils/DataLoader.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens["token1"] = tokens["token1"].str.lower()


LOAD RUG PULL INFO


In [13]:
def count_hops_from_source(source_address, paths: list):
    source_address = source_address.lower()
    if not source_address in paths:
        return 0
    return len(paths[paths.index(source_address):])


def get_nodes_by_labels(root_label, target_labels, cluster: LightCluster):
    root_label_nodes = []
    target_label_nodes = dict()
    for node in cluster.nodes.values():
        if root_label in node.labels and not any(label in LightNode.LightNodeLabel.SKIP_LABELS for label in node.labels):
            root_label_nodes.append(node)
        for l in node.labels:
            if l in target_labels:
                nodes = []
                if l in target_label_nodes.keys():
                    nodes = target_label_nodes[l]
                nodes.append(node)
                target_label_nodes[l] = nodes
    return root_label_nodes, target_label_nodes


def find_impacts(root_node, target_nodes):
    satisfied_nodes = []
    hops = []
    for node in target_nodes:
        if root_node.address in node.path:
            satisfied_nodes.append(node)
            hops.append(count_hops_from_source(root_node.address, node.path))
    return satisfied_nodes, hops


def impact_analysis(root_label, target_labels, cluster: LightCluster):
    root_label_nodes, target_label_nodes = get_nodes_by_labels(root_label, target_labels, cluster)
    results = []
    for root in root_label_nodes:
        record = LightNode.LightNode.to_sort_dict(root)
        impacts = []
        for tl in target_label_nodes.keys():
            tgn = target_label_nodes.get(tl)
            satisfied_nodes, hops = find_impacts(root, tgn)
            impacts.append({"type": tl, "count": len(satisfied_nodes), "avg_distance": sum(hops) / len(hops) if len(hops) > 0 else 0})
        record["impacts"] = impacts
        results.append(record)
    return results




In [6]:
group_id = 3
cluster = LightCluster.LightCluster(group_id)
cluster.load(cluster_path)

LOAD EXISTING CLUSTER


In [14]:
results = impact_analysis(LightNode.LightNodeLabel.BIG_CONNECTOR, [LightNode.LightNodeLabel.BIG_CONNECTOR, LightNode.LightNodeLabel.SCAMMER, LightNode.LightNodeLabel.BIG_WASHTRADER], cluster)
display(results)

[{'address': '0x4e756eb534caf866f911117f9c5b9927c11927cf',
  'valid_neighbours': 152,
  'normal_txs_len': 438,
  'labels': 'withdrawer;depositor;big_connector',
  'impacts': [{'type': 'scammer', 'count': 0, 'avg_distance': 0},
   {'type': 'big_washtrader', 'count': 48, 'avg_distance': 2.0},
   {'type': 'big_connector', 'count': 9, 'avg_distance': 1.8888888888888888}]},
 {'address': '0x167c6a6948606e410cd79baac3be52d949cbcd3c',
  'valid_neighbours': 135,
  'normal_txs_len': 2364,
  'labels': 'big_washtrader;over_limit_2_but_accept;depositor;withdrawer;big_connector',
  'impacts': [{'type': 'scammer', 'count': 127, 'avg_distance': 2.0},
   {'type': 'big_washtrader', 'count': 8, 'avg_distance': 1.875},
   {'type': 'big_connector', 'count': 2, 'avg_distance': 1.5}]},
 {'address': '0xfa19e5e2ca003c0195859b10296867a2a10514f9',
  'valid_neighbours': 58,
  'normal_txs_len': 698,
  'labels': 'big_washtrader;over_limit_2_but_accept;big_connector',
  'impacts': [{'type': 'scammer', 'count': 41, '